In [9]:
import json
import boto3
import time

cars_filename         = 'car_items.csv'
users_filename        = 'users.csv'
interactions_filename = 'interactions.csv'

dataset_group_arn = 'arn:aws:personalize:us-east-1:355151823911:dataset-group/car-dg10'

MAX_WAIT_TIME = 60*60 # 1 hour

role_arn = 'arn:aws:iam::355151823911:role/PersonalizeS3Role'

In [10]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [12]:
def make_campaign(recipe_arn, name, dataset_group_arn):
    print('Entered make_campaign for {}'.format(name))
    create_solution_response = personalize.create_solution(
        name = name,
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn
    )
    solution_arn = create_solution_response['solutionArn']
    print('created solution: {}'.format(solution_arn))

    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn
    )
    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print('Waiting for solution version to become active...')
    latest_time = time.time() + MAX_WAIT_TIME
    while time.time() < latest_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response['solutionVersion']['status']
        print('SolutionVersion: {}'.format(status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            if status == 'CREATE FAILED':
                print('*** Solution Version Create failed ***')
            break

        time.sleep(60)
    get_solution_metrics_response = personalize.get_solution_metrics(
        solutionVersionArn = solution_version_arn
    )

    print(json.dumps(get_solution_metrics_response, indent=2))    

    create_campaign_response = personalize.create_campaign(
        name = name,
        solutionVersionArn = solution_version_arn,
        minProvisionedTPS = 1
    )
    campaign_arn = create_campaign_response['campaignArn']
    print('Waiting for campaign to become active...')

    latest_time = time.time() + MAX_WAIT_TIME
    while time.time() < latest_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = describe_campaign_response['campaign']['status']
        print('Campaign: {}'.format(status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            break

        time.sleep(60)
        
    print('Exiting make_campaign for {}'.format(name))
    return(solution_arn, solution_version_arn, campaign_arn)

In [ ]:
make_campaign('arn:aws:personalize:::recipe/aws-hrnn', 'car-hrnn', dataset_group_arn)

Entered make_campaign for car-hrnn
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn
Waiting for solution version to become active...
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersi

('arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn',
 'arn:aws:personalize:us-east-1:355151823911:solution/car-hrnn/d46b023a',
 'arn:aws:personalize:us-east-1:355151823911:campaign/car-hrnn')

In [ ]:
make_campaign('arn:aws:personalize:::recipe/aws-sims', 'car-sims', dataset_group_arn)

Entered make_campaign for car-sims
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-sims
Waiting for solution version to become active...
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


ResourceInUseException: An error occurred (ResourceInUseException) when calling the GetSolutionMetrics operation: Cannot get metrics for solution since it is not in ACTIVE state.

In [37]:
make_campaign('arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking', dataset_group_arn)

Entered make_campaign for car-personalized-ranking
Waiting for solution version to become active...
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
Solu

('arn:aws:personalize:us-east-1:355151823911:solution/car-personalized-ranking',
 'arn:aws:personalize:us-east-1:355151823911:solution/car-personalized-ranking/4e52d628',
 'arn:aws:personalize:us-east-1:355151823911:campaign/car-personalized-ranking')

In [27]:
#make_campaign('arn:aws:personalize:::recipe/aws-hrnn-metadata', 'car-hrnn-metadata', dataset_group_arn)

In [13]:
from multiprocessing import Process

def make_campaigns_in_parallel(campaigns, dg_arn):
    jobs = []
    for i in campaigns:
        p = Process(target = make_campaign, args=(i[0], i[1], dg_arn))
        jobs.append(p)
        
    for p in jobs:
        p.start()
        
    for p in jobs:
        p.join():

In [ ]:
campaigns = [['arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking'],
             ['arn:aws:personalize:::recipe/aws-sims',                 'car-sims'],
             ['arn:aws:personalize:::recipe/aws-popularity-count',     'car-popularity-count'],
             ['arn:aws:personalize:::recipe/aws-hrnn-metadata',        'car-hrnn-metadata'],
             ['arn:aws:personalize:::recipe/aws-hrnn',                 'car-hrnn']
            ]
print(dataset_group_arn)
make_campaigns_in_parallel(campaigns, dataset_group_arn)

arn:aws:personalize:us-east-1:355151823911:dataset-group/car-dg10
['arn:aws:personalize:::recipe/aws-personalized-ranking', 'car-personalized-ranking']
Entered make_campaign for car-personalized-ranking
created solution: arn:aws:personalize:us-east-1:355151823911:solution/car-personalized-ranking
Waiting for solution version to become active...
SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionV

In [11]:
def make_just_campaign(recipe_arn, solution_version_arn, name, dataset_group_arn):
    print('Waiting for solution version to become active...')
    while time.time() < MAX_WAIT_TIME:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response['solutionVersion']['status']
        print('SolutionVersion: {}'.format(status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            if status == 'CREATE FAILED':
                print('*** Solution Version Create failed ***')
            break

        time.sleep(60)
        
    get_solution_metrics_response = personalize.get_solution_metrics(
        solutionVersionArn = solution_version_arn
    )

    print(json.dumps(get_solution_metrics_response, indent=2))    

    print('Entered make_just_campaign for {}'.format(name))
    create_campaign_response = personalize.create_campaign(
        name = name,
        solutionVersionArn = solution_version_arn,
        minProvisionedTPS = 1
    )
    campaign_arn = create_campaign_response['campaignArn']
    print('Waiting for campaign to become active...')
    while time.time() < MAX_WAIT_TIME:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = describe_campaign_response['campaign']['status']
        print('Campaign: {}'.format(status))

        if status == 'ACTIVE' or status == 'CREATE FAILED':
            break

        time.sleep(60)
        
    print('Exiting make_just_campaign for {}'.format(name))
    return(solution_version_arn, campaign_arn)

In [15]:
#make_just_campaign('arn:aws:personalize:::recipe/aws-popularity-count', 
#                   'arn:aws:personalize:us-east-1:355151823911:solution/car-popularity/3bcc545a',
#                   'car-popularity', dataset_group_arn)